<center><img src="./imge/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 20.基于Jetson Nano识别和定位图像中感兴趣的对象

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：20.基于Jetson Nano识别和定位图像中感兴趣的对象.ipynb <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：基于Jetson Nano识别和定位图像中感兴趣的对象<br>

In [ ]:
import cv2
import jetson.inference
import jetson.utils
import time
import numpy as np

In [1]:
timeStamp= time.time()
fpsFiltered=0

NameError: name 'time' is not defined

In [ ]:
from jetcam.utils import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

frame_img = widgets.Image(format='jpeg', width=1280, height=720)
display(frame_img)

In [ ]:
net=jetson.inference.detectNet('ssd-mobilenet-v2',threshold=.5)
dispW=1280
dispH=720
flip=4
font=cv2.FONT_HERSHEY_SIMPLEX

# Gstreamer code for improvded Raspberry Pi Camera Quality
camSet='nvarguscamerasrc wbmode=3 tnr-mode=2 tnr-strength=1 ee-mode=2 ee-strength=1 ! video/x-raw(memory:NVMM), width=3264, height=2464, format=NV12, framerate=21/1 ! nvvidconv flip-method='+str(flip)+' ! video/x-raw, width='+str(dispW)+', height='+str(dispH)+', format=BGRx ! videoconvert ! video/x-raw, format=BGR ! videobalance contrast=1.5 brightness=-.2 saturation=1.2 ! appsink'
cam=cv2.VideoCapture(camSet)

while True:
    _,img = cam.read()
    height= img.shape[0]
    width=  img.shape[1]
    
    frame=cv2.cvtColor(img,cv2.COLOR_BGR2RGBA).astype(np.float32)
    frame=jetson.utils.cudaFromNumpy(frame)
    detections=net.Detect(frame,width,height)
    
    for detect in detections:
        #print(detect)
        ID=detect.ClassID
        top=int(detect.Top)
        left=int(detect.Left)
        bottom=int(detect.Bottom)
        right=int(detect.Right)
        item=net.GetClassDesc(ID)
        tk=1
        if item == 'cat': # 识别猫
            tk = -1
        cv2.rectangle(img,(left,top),(right,bottom),(0,255,0),tk)
        cv2.putText(img,item,(left,top+20),font,.75,(0,0,255),2)
        
        #print(item,top,left,bottom,right)
    
    #print(str(round(fps,1))+' fps ')
    dt=time.time()-timeStamp
    timeStamp=time.time()
    fps=1/dt
    fpsFiltered=.9*fpsFiltered + .1*fps
    cv2.putText(img,str(round(fpsFiltered,1)) + ' fps ',(0,30),font,1,(0,0,255),2)
    frame_img.value = bgr8_to_jpeg(img)